# Build an Agent

By themselves, language models can't take actions - they just output text.
A big use case for LangChain is creating **agents**.
[Agents](/docs/concepts/agents) are systems that use [LLMs](/docs/concepts/chat_models) as reasoning engines to determine which actions to take and the inputs necessary to perform the action.
After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via [tool-calling](/docs/concepts/tool_calling).

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## End-to-end agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!

In [1]:
!pip install -U langchain-community langgraph langchain-openai tavily-python langgraph-checkpoint-sqlite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 15.3 MB/s eta 0:00:00


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
cd '/content/drive/MyDrive/Courses/AI/Lecture13'

/content/drive/MyDrive/Courses/AI/Lecture13


In [18]:
# Import relevant functionality
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv

load_dotenv("api_keys.env")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Create the agent
memory = MemorySaver()
model = ChatOpenAI(model_name="gpt-4", temperature=0.7)
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [21]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Juan! and i live in Medellín")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im Juan! and i live in Medellín
================================== Ai Message ==================================

Hi Juan! How can I assist you today?


In [22]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_RJAYvdN4OKoaAx5EOhLu23f3)
 Call ID: call_RJAYvdN4OKoaAx5EOhLu23f3
  Args:
    query: current weather in Medellín
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather for Medellin, Colombia - Time and Date", "url": "https://www.timeanddate.com/weather/colombia/medellin", "content": "Weather in Medellin, Colombia\n\nScattered clouds.\n\nFeels Like: 64 °FForecast: 75 / 62 °FWind: 3 mph ↑ from North\n\nLocation: | Rionegro / J. M. Cordova\nCurrent Time: | May 5, 2025 at 10:47:06 am\nLatest Report: | May 5, 2025 at 10:00 am\nVisibility: | N/A\nPressure: | 30.33 \"Hg(25.50 \"Hg at 1479m altitude)\nHumidity: | 73%\nDew Point: | 55 °F\nUpcoming 5 hours [...] * Updated Monday,

## Setup

### Jupyter Notebook

This guide (and most of the other guides in the documentation) uses [Jupyter notebooks](https://jupyter.org/) and assumes the reader is as well. Jupyter notebooks are perfect interactive environments for learning how to work with LLM systems because oftentimes things can go wrong (unexpected output, API down, etc), and observing these cases is a great way to better understand building with LLMs.

This and other tutorials are perhaps most conveniently run in a Jupyter notebook. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation

To install LangChain run:

For more details, see our [Installation guide](/docs/how_to/installation).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

### Tavily

We will be using [Tavily](/docs/integrations/tools/tavily_search) (a search engine) as a tool.
In order to use it, you will need to get and set an API key:

```bash
export TAVILY_API_KEY="..."
```

Or, if in a notebook, you can set it with:

```python
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()
```

## Define tools

We first need to create the tools we want to use. Our main tool of choice will be [Tavily](/docs/integrations/tools/tavily_search) - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.


In [24]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Medellín")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in Medellín', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Medellín', 'region': 'Antioquia', 'country': 'La Colombie', 'lat': 6.2914, 'lon': -75.5361, 'tz_id': 'America/Bogota', 'localtime_epoch': 1746460224, 'localtime': '2025-05-05 10:50'}, 'current': {'last_updated_epoch': 1746459900, 'last_updated': '2025-05-05 10:45', 'temp_c': 22.2, 'temp_f': 72.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 140, 'wind_dir': 'SE', 'pressure_mb': 1023.0, 'pressure_in': 30.21, 'precip_mm': 0.45, 'precip_in': 0.02, 'humidity': 78, 'cloud': 50, 'feelslike_c': 24.6, 'feelslike_f': 76.3, 'windchill_c': 19.7, 'windchill_f': 67.4, 'heatindex_c': 19.7, 'heatindex_f': 67.4, 'dewpoint_c': 14.3, 'dewpoint_f': 57.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 5.3, 'gust_mph': 6.7, 'gust_kph': 10.8}}", 'score': 0.9325906}, {'title': 'Medel

## Using Language Models

Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4"}}} />


In [26]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4", temperature=0.7)

You can call the language model by passing in a list of messages. By default, the response is a `content` string.

In [27]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [28]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [29]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [30]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Medellín?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Medellín'}, 'id': 'call_Gu2459JRDc8Y1Fd9RqH5Jehb', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create the agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using [LangGraph](/docs/concepts/architecture/#langgraph) to construct the agent.
Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.


Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [31]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

We can now run the agent with a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions). Note that the agent will return the **final** state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [32]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='aadd05be-9166-4ef1-859e-9c98858404f5'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BTsOvbilNquF2be5QkrK42U4qRL2w', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run-3590a20b-d129-4c4f-9ccf-13115eec44ee-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

Let's now try it out on an example where it should be invoking the tool

In [34]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Medellín?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in Medellín?', additional_kwargs={}, response_metadata={}, id='10d487d0-8e99-41bb-8d5c-e5261f31d925'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RNNdFrOszKQz8WnNTTntJ1rX', 'function': {'arguments': '{\n  "query": "current weather in Medellín"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 90, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BTsP5txEjWHGimzumyo1oQbucNLmf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-27446ebf-366c-48fb-a9a8-7acd1cac3a72-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query

We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.

## Streaming Messages

We've seen how the agent can be called with `.invoke` to get  a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [35]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Medellin?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in Medellin?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_os8U9fg4eioOxTJOrmfnqGJI)
 Call ID: call_os8U9fg4eioOxTJOrmfnqGJI
  Args:
    query: current weather in Medellin
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather for Medellin, Colombia - Time and Date", "url": "https://www.timeanddate.com/weather/colombia/medellin", "content": "Weather in Medellin, Colombia ; May 5, 2025 at 5:00 am · N/A · 30.27 \"Hg (25.38 \"Hg at 1479m altitude) · 94% · 55 °F", "score": 0.9014448}, {"title": "Medellin weather in May 2025 - Weather25.com", "url": "https://www.weather25.com/south-america/colombia/sucre/medellin?page=month&month=May", "content": "Medellin weather in May 2025\n\nThe average weather in Medellin in May\n\nThe temperatu

## Streaming tokens

In addition to streaming back messages, it is also useful to stream back tokens.
We can do this by specifying `stream_mode="messages"`.


::: note

Below we use `message.text()`, which requires `langchain-core>=0.3.37`.

:::

In [36]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Medellín?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| Med|ell|ín| is| partly| cloudy| with| a| temperature| of| |22|.|2|°C| (|72|.|0|°F|).| The| wind| is| coming| from| the| southeast| at| a| speed| of| |3|.|6| k|ph| (|2|.|2| mph|).| The| humidity| level| is| at| |78|%| and| there|'s| a| slight| precipitation| of| |0|.|45|mm|.| The| visibility| is| about| |10|.|0| km|.| Please| note| that| this| information| is| for| the| date| of| May| |5|,| |202|5|,| and| the| weather| conditions| may| have| changed|.|

## Adding in memory

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [37]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [38]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [39]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Juan!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Juan! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 85, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BTsQWKwwo9LukQcFoYHFbqjIZMHh0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run-b5c89353-5df2-40dd-9738-99bc4521d342-0', usage_metadata={'input_tokens': 85, 'output_tokens': 12, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [40]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Juan.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 108, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BTsQbS7Cezl7Q5FNVVgvmtIYGbZUz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run-d63d824a-ced4-48c1-b564-13289d4cb873-0', usage_metadata={'input_tokens': 108, 'output_tokens': 7, 'total_tokens': 115, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If you want to start a new conversation, all you have to do is change the `thread_id` used

In [41]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 86, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BTsQmQgy1ZsotiFSbooH2QS8emHWb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run-49aea9ac-b0c0-4253-8347-671647f3e123-0', usage_metadata={'input_tokens': 86, 'output_tokens': 40, 'total_tokens': 126, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
---

## Conclusion

That's a wrap! In this quick start we covered how to create a simple agent.
We've then shown how to stream back a response - not only with the intermediate steps, but also tokens!
We've also added in memory so you can have a conversation with them.
Agents are a complex topic with lots to learn!

For more information on Agents, please check out the [LangGraph](/docs/concepts/architecture/#langgraph) documentation. This has it's own set of concepts, tutorials, and how-to guides.